In [1]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import pandas_gbq
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("styler.format.thousands", ",")

In [2]:
#Change the date to the previous day
query = """
SELECT *, "Regular Season" as Category FROM `perceptive-ivy-290216.nba_api_box_player_track.all_time_reg_season`
UNION ALL
SELECT *, "Playoffs" as Category FROM `perceptive-ivy-290216.nba_api_box_player_track.all_time_playoff` 
"""
project_id = "perceptive-ivy-290216"
df_bq_misc_player = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [3]:
#USE FOR ALL TIME DATA
df_bq3=df_bq_misc_player.copy()

#USE FOR SINGLE SEASON PLOTS
# current_year="2023-24"
# df_bq2=df_bq_reg_season[df_bq_reg_season["Season"]==current_year].copy()

df_bq3=df_bq3.fillna(0)
types="Regular Season"
# types="Playoffs"
df_bq2=df_bq3[df_bq3["Category"]==types]
df_bq2.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,SPD,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,AST,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,FG_PCT,DFGM,DFGA,DFG_PCT,Season,GAME_ID_1,GAME_DATE,MATCHUP,SEASON_1,Category
24394,0022200030,1610612753,ORL,Orlando,1629626,Bol Bol,,,14:52,4.42,1.17,2,6,7,19,0,0,11,0,1,3,0.333,2,4,0.500,0.428,1,1,1.000,2022-23,0022200030,2022-10-22,BOS @ ORL,2022-23,Regular Season
24395,0022200093,1610612753,ORL,Orlando,1629626,Bol Bol,F,,29:44,4.30,2.28,5,8,12,28,0,0,16,0,6,7,0.857,1,3,0.333,0.699,4,6,0.667,2022-23,0022200093,2022-10-30,ORL @ DAL,2022-23,Regular Season
24396,0022200888,1610612753,ORL,Orlando,1629626,Bol Bol,,,16:10,3.96,1.16,1,4,5,12,1,0,7,1,0,1,0.000,0,3,0.000,0.000,1,2,0.500,2022-23,0022200888,2023-02-23,DET @ ORL,2022-23,Regular Season
24397,0022200105,1610612753,ORL,Orlando,1629626,Bol Bol,F,,30:53,4.30,2.40,3,13,16,36,0,0,22,1,3,4,0.750,1,2,0.500,0.667,6,9,0.667,2022-23,0022200105,2022-11-01,ORL @ OKC,2022-23,Regular Season
24398,0022200505,1610612753,ORL,Orlando,1629626,Bol Bol,F,,21:59,4.36,1.68,2,5,7,19,0,0,10,0,3,4,0.750,1,3,0.333,0.570,2,2,1.000,2022-23,0022200505,2022-12-27,LAL @ ORL,2022-23,Regular Season


In [4]:
#Add Game Number for trended values plotting
df_bq2["Game Number"] = df_bq2.groupby("PLAYER_NAME")["GAME_DATE"].rank(method="dense", ascending=True)
df_bq2["Game Number"]=df_bq2["Game Number"].astype(int)

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1984/3610659743.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bq2["Game Number"] = df_bq2.groupby("PLAYER_NAME")["GAME_DATE"].rank(method="dense", ascending=True)
/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1984/3610659743.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bq2["Game Number"]=df_bq2["Game Number"].astype(int)


In [7]:
#Get cumulative stat for each player
df_bq2['Total SPD'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['SPD'].cumsum()
df_bq2['Total DIST'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['DIST'].cumsum()
df_bq2['Total ORBC'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['ORBC'].cumsum()
df_bq2['Total DRBC'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['DRBC'].cumsum()
df_bq2['Total RBC'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['RBC'].cumsum()
df_bq2['Total TCHS'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['TCHS'].cumsum()
df_bq2['Total SAST'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['SAST'].cumsum()
df_bq2['Total FTAST'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FTAST'].cumsum()
df_bq2['Total PASS'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PASS'].cumsum()
df_bq2['Total AST'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['AST'].cumsum()
df_bq2['Total CFGM'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['CFGM'].cumsum()
df_bq2['Total CFGA'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['CFGA'].cumsum()
df_bq2['Total UFGM'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['UFGM'].cumsum()
df_bq2['Total UFGA'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['UFGA'].cumsum()
df_bq2['Total DFGM'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['DFGM'].cumsum()
df_bq2['Total DFGA'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['DFGA'].cumsum()


df_bq2['Total CFG_PCT']=df_bq2['Total CFGM']/df_bq2['Total CFGA']
df_bq2['Total UFG_PCT']=df_bq2['Total UFGM']/df_bq2['Total UFGA']
df_bq2['Total DFG_PCT']=df_bq2['Total DFGM']/df_bq2['Total DFGA']

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1984/1886451995.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bq2['Total SPD'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['SPD'].cumsum()
/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1984/1886451995.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bq2['Total DIST'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['DIST'].cumsum()
/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1984/18

In [10]:
#Get Totals For Each Category
df_totals=pd.DataFrame(df_bq2.groupby(by=['PLAYER_ID','PLAYER_NAME'])[[
       'SPD', 'DIST','ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM',
       'CFGA', 'UFGM', 'UFGA','DFGM', 'DFGA']].sum()).reset_index()

df_totals['CFG_PCT']=df_totals['CFGM']/df_totals['CFGA']
df_totals['UFG_PCT']=df_totals['UFGM']/df_totals['UFGA']
df_totals['DFG_PCT']=df_totals['DFGM']/df_totals['DFGA']

#Get Game Counts for Each Player
df_count=pd.DataFrame(df_bq2.groupby(by=['PLAYER_ID','PLAYER_NAME'])[["GAME_ID"]].count()).reset_index()
df_count.columns=["PLAYER_ID","PLAYER_NAME","GP"]
df_totals=df_totals.merge(df_count, left_on=["PLAYER_NAME","PLAYER_ID"],right_on=["PLAYER_NAME","PLAYER_ID"])
df_totals.head()

,PLAYER_ID,PLAYER_NAME,SPD,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,AST,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,CFG_PCT,UFG_PCT,DFG_PCT,GP
0,708,Kevin Garnett,572.10,178.03,290,1078,1314,5393,20,11,4417,221,121,315,227,454,211,356,0.384127,0.5,0.592697,178
1,951,Ray Allen,309.03,136.40,106,331,434,2355,17,11,1613,143,67,155,173,388,101,128,0.432258,0.445876,0.789062,75
2,959,Steve Nash,67.41,23.39,17,52,67,773,11,4,625,86,13,36,23,58,14,14,0.361111,0.396552,1.0,40
3,965,Derek Fisher,327.49,95.58,60,198,253,2155,14,14,1623,117,42,136,99,225,42,53,0.308824,0.44,0.792453,82
4,977,Kobe Bryant,425.30,214.14,301,671,953,5749,37,35,3176,419,373,1027,322,872,122,172,0.363194,0.369266,0.709302,149


In [20]:
sort_category="DIST"
# sort_category=[ 'SPD', 'DIST', 'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM',
#        'CFGA', 'CFG_PCT', 'UFGM', 'UFGA', 'UFG_PCT', 'FG_PCT', 'DFGM', 'DFGA',
#        'DFG_PCT']

In [12]:
#Sort by a category and then rank it. This will help get the Top Players in a category
df_totals['RK_Category']=df_totals[sort_category].rank(method="dense", ascending=False)
df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
top=df_totals.sort_values(by=sort_category, ascending=False).head(10)
top

,PLAYER_ID,PLAYER_NAME,SPD,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,AST,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,CFG_PCT,UFG_PCT,DFG_PCT,GP,RK_Category
395,203084,Harrison Barnes,3815.16,2113.40,2540,5859,8147,39171,233,96,26395,1566,1760,3717,2641,5888,1536,2259,0.4735,0.448539,0.679947,888,1
484,203484,Kentavious Caldwell-Pope,3659.97,1881.82,1396,3687,4997,32182,253,127,21545,1638,1102,2604,2450,5685,1022,1393,0.423195,0.430959,0.733668,901,2
352,202699,Tobias Harris,3540.60,2036.68,2797,7482,9940,48592,339,166,33585,2188,2751,5471,2816,6092,1846,2705,0.502833,0.462246,0.68244,882,3
360,202709,Cory Joseph,3533.86,1341.59,1259,3118,4310,35318,324,148,27988,2404,840,1831,1257,2925,838,1236,0.458766,0.429744,0.677994,875,4
474,203471,Dennis Schroder,3511.44,1669.56,1395,3603,4957,53367,660,302,39013,4009,1801,4029,2378,5597,977,1394,0.447009,0.42487,0.700861,868,5
278,201988,Patty Mills,3484.26,1290.88,699,1901,2570,29208,332,117,21459,1787,608,1424,1891,4518,502,672,0.426966,0.418548,0.747024,893,6
507,203507,Giannis Antetokounmpo,3444.95,1951.52,4167,10088,13694,60405,504,179,39516,4164,4349,7143,2918,6074,1768,3289,0.608848,0.480408,0.537549,863,7
247,201942,DeMar DeRozan,3427.20,2110.01,2165,5630,7545,52223,650,198,30784,4255,3763,7869,3261,6986,1354,1888,0.478206,0.466791,0.717161,876,8
486,203486,Mason Plumlee,3413.91,1301.62,4272,6507,10488,29556,168,112,22221,1934,1812,3191,814,1211,2072,3550,0.567847,0.672172,0.583662,841,9
394,203083,Andre Drummond,3410.85,1574.19,7148,10234,16767,32224,97,68,20280,1070,3411,6621,1153,1822,2448,4155,0.515179,0.632821,0.58917,883,10


In [13]:
#Get Player Names of the Top Category we have selected
names=top.PLAYER_NAME.to_list()

In [18]:
#Get the DF with Player Names from the top category
df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]
df_top.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,SPD,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,AST,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,FG_PCT,DFGM,DFGA,DFG_PCT,Season,GAME_ID_1,GAME_DATE,MATCHUP,SEASON_1,Category,Game Number,Total SPD,Total DIST,Total ORBC,Total DRBC,Total RBC,Total TCHS,Total SAST,Total FTAST,Total PASS,Total AST,Total CFGM,Total CFGA,Total UFGM,Total UFGA,Total DFGM,Total DFGA,Total CFG_PCT,Total UFG_PCT,Total DFG_PCT
28628,0022200309,1610612765,DET,Detroit,202709,Cory Joseph,,,21:58,4.47,1.78,3,2,5,55,0,1,43,3,1,1,1.00,4,7,0.571,0.625,1,1,1.0,2022-23,0022200309,2022-11-29,NYK @ DET,2022-23,Regular Season,727,3104.62,1219.21,1175,2864,3978,32137,303,127,25393,2152,802,1734,1137,2626,756,1120,0.462514,0.432978,0.675
28629,0022201020,1610612765,DET,Detroit,202709,Cory Joseph,,,32:18,4.27,2.48,3,7,10,67,0,1,49,5,0,4,0.00,8,10,0.800,0.570,3,5,0.6,2022-23,0022201020,2023-03-13,IND @ DET,2022-23,Regular Season,766,3249.69,1269.40,1201,2977,4116,33504,311,140,26496,2273,823,1784,1188,2747,790,1165,0.461323,0.432472,0.678112
28630,0022200957,1610612765,DET,Detroit,202709,Cory Joseph,,,23:40,4.32,1.84,0,7,7,46,0,0,35,3,0,0,0.00,2,3,0.667,0.667,0,0,0.0,2022-23,0022200957,2023-03-04,DET @ CLE,2022-23,Regular Season,761,3228.14,1258.11,1194,2955,4087,33198,308,138,26260,2249,816,1767,1166,2712,783,1154,0.4618,0.429941,0.67851
28631,0022200977,1610612765,DET,Detroit,202709,Cory Joseph,,,31:36,4.15,2.36,1,6,7,64,0,0,49,4,3,4,0.75,3,7,0.429,0.545,2,2,1.0,2022-23,0022200977,2023-03-06,POR @ DET,2022-23,Regular Season,762,3232.29,1260.47,1195,2961,4094,33262,308,138,26309,2253,819,1771,1169,2719,785,1156,0.462451,0.429937,0.679066
28632,0022200186,1610612765,DET,Detroit,202709,Cory Joseph,,,10:41,4.61,0.89,0,2,2,25,1,0,20,2,1,1,1.00,1,3,0.333,0.500,0,0,0.0,2022-23,0022200186,2022-11-12,BOS @ DET,2022-23,Regular Season,720,3082.54,1211.70,1166,2854,3960,31914,302,125,25223,2133,794,1720,1125,2598,748,1111,0.461628,0.433025,0.673267


In [15]:
#Get Trended Data Point 

In [21]:
fig = px.scatter(
  df_top, 
  x="Game Number", 
  y="Total {}".format(sort_category),
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>{} All Time Trended Player {} vs. Game Number</b>".format(types,sort_category),
  template="plotly_white",
  hover_data=['GAME_DATE','MATCHUP','TEAM_ABBREVIATION', 'Season',
       'PLAYER_NAME', 'DIST', 'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM',
       'CFGA', 'CFG_PCT', 'UFGM', 'UFGA', 'UFG_PCT', 'FG_PCT', 'DFGM', 'DFGA','DFG_PCT'], 
  height=800, 
  width=1200,
  )

fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.update_traces(marker={'size': 3})

fig

In [ ]:
fig.write_html("{} All Time Trended Player {} vs. Game Number.html".format(types,sort_category),full_html=False, include_plotlyjs='cdn')
# fig.write_html("Trended Player {} vs. Game Number For The {} Season.html".format(sort_category,current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#2 variable scatter plot

In [25]:
cat1="TCHS"
cat2="PASS"

fig_scatter = px.scatter(
  df_totals, 
  x=cat1, 
  y=cat2,
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>{} All Time Player {} vs. {}</b>".format(types,cat1,cat2),
  template="plotly_white",
  hover_data=['PLAYER_NAME', 'DIST', 'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM',
       'CFGA', 'CFG_PCT', 'UFGM', 'UFGA', 'UFG_PCT', 'DFGM', 'DFGA','DFG_PCT'], 
  height=800, 
  width=1200,
  )

fig_scatter.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter.update_traces(marker={'size': 10})

fig_scatter

In [25]:
fig_scatter.write_html("{} All Time Player {} vs. {}.html".format(types,cat1,cat2),full_html=False, include_plotlyjs='cdn')
# fig_scatter.write_html("Player {} vs. {} For The {} Season.html".format(cat1,cat2, current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Bar Chart For Category

In [26]:
fig_bar = px.bar(
  df_totals.sort_values(by=sort_category, ascending=False).head(25),
  x='PLAYER_NAME', 
  y=sort_category,
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>{} All Time Player {}</b>".format(types,sort_category),
  template="plotly_white",
  hover_data=['PLAYER_NAME', 'DIST', 'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM',
       'CFGA', 'CFG_PCT', 'UFGM', 'UFGA', 'UFG_PCT', 'DFGM', 'DFGA','DFG_PCT'], 
  height=800, 
  width=1200,
  )

fig_bar.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_bar.update_traces(marker_color='#0047ab')
fig_bar

In [ ]:
fig_bar.write_html("{} All Time Player {}</b>".format(types,sort_category),full_html=False, include_plotlyjs='cdn')
# fig_bar.write_html("Player {} For The {} {}</b>".format(sort_category, current_year, types),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOTS TOGETHER

In [27]:
sort_categories=[ 'SPD', 'DIST', 'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM',
       'CFGA', 'CFG_PCT', 'UFGM', 'UFGA', 'UFG_PCT', 'FG_PCT', 'DFGM', 'DFGA',
       'DFG_PCT']

In [ ]:
#TRENDED

In [244]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top.PLAYER_NAME.to_list()
    df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]

    fig_multi = px.scatter(
    df_top, 
    x="Game Number", 
    y="Total {}".format(i),
    color='PLAYER_NAME',
    color_discrete_sequence=px.colors.qualitative.G10,
    title="<b>{} All Time Trended Player {} vs. Game Number</b>".format(types, i),
    template="plotly_white",
  hover_data=['PLAYER_NAME', 'DIST', 'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM',
       'CFGA', 'CFG_PCT', 'UFGM', 'UFGA', 'UFG_PCT', 'DFGM', 'DFGA','DFG_PCT'], 
        #          'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        # 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
        # 'Game Number'
        ], 
    height=800, 
    width=1200,
    )

    fig_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_multi.update_traces(marker={'size': 3})
    fig_multi.write_html("{} All Time Trended Player {} vs. Game Number.html".format(types, i),full_html=False, include_plotlyjs='cdn')
    # fig_multi.write_html("Trended Player {} vs. Game Number For The {} Season.html".format(i, current_year),full_html=False, include_plotlyjs='cdn')    

In [29]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top.PLAYER_NAME.to_list()
    df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]

    fig_bar_multi = px.bar(
    df_totals.sort_values(by=i, ascending=False).head(25),
    x='PLAYER_NAME', 
    y=i,
    color='PLAYER_NAME',
    color_discrete_sequence=px.colors.qualitative.G10,
    title="<b>{} All Time Player {}</b>".format(types,i),
    template="plotly_white",
  hover_data=['PLAYER_NAME', 'DIST', 'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM',
       'CFGA', 'CFG_PCT', 'UFGM', 'UFGA', 'UFG_PCT', 'DFGM', 'DFGA','DFG_PCT'], 
    height=800, 
    width=1200,
    )

    fig_bar_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_bar_multi.update_traces(marker_color='#0047ab')
    fig_bar_multi.write_html("{} All Time Player {}.html".format(types,i),full_html=False, include_plotlyjs='cdn')
# fig_bar_multi.write_html("Player {} For The {} {}</b>".format(sort_category, current_year, types),full_html=False, include_plotlyjs='cdn')